In [0]:
import base64

import databricks.sdk
from databricks.sdk.service.workspace import ImportFormat
import nbformat
import numpy as np 

import pandas as pd

In [0]:
import dspy
lm = dspy.LM('databricks/databricks-meta-llama-3-3-70b-instruct')
dspy.settings.configure(lm=lm)

w = databricks.sdk.WorkspaceClient()

In [0]:
def get_code_from_notebooks(workspace_client, notebook_path):
   notebook = workspace_client.workspace.export(notebook_path, 
                                    format=ImportFormat.JUPYTER)
   ipynb = base64.decodebytes(notebook.content.encode('ascii')).decode("utf-8")
   notebook = nbformat.reads(ipynb, as_version=4)
   code_dict = {}
   x=0
   for cell in notebook.cells:
      if cell.cell_type == 'code' or cell.cell_type == 'markdown':
         code_dict[x] = cell.source
         x+=1
   context = ''.join(code_dict.values())
   return context

In [0]:
def evaluate_responses(context, module_dict):
  answer_list = []
  for k,module in module_dict.items():
    response = module(text=context)
    response_dict = {
      'score': response.score,
      'reasoning': response.reasoning,
      'code_snippet': response.code_snippet
    }
    # print(response_dict)
    answer_list.append(response_dict)
  answer_list_df = pd.DataFrame(answer_list)

  return answer_list_df

In [0]:
def get_error_and_answer_dict(context_path, module_dict, human_answers_list, table_name, section):
  
  context = get_code_from_notebooks(w, context_path)
  answers = evaluate_responses(context, module_dict)
  answers['candidate'] = candidate[0]
  answers['secton'] = section
  answers['score'] = answers['score'].astype('float64')
  spark.createDataFrame(answers).write.mode("append").saveAsTable(table_name)

  model_asnwers_list = answers['score'].tolist()
  diff = [human_answers_list[i] - model_asnwers_list[i] for i in range(len(human_answers_list))]
  
  abs_error = np.abs(diff).sum()
  inaccurate_predictions = len([x for x in diff if x != 0])

  error_dict = {'abs_error': abs_error, 'inaccurate_predictions': inaccurate_predictions, 'context': diff}

  return {
    'error_dict': error_dict,
    'answers_dict': answers
  }

In [0]:
def evaluate_model(diff):
  unmatched_elements = [i for i in diff if i != 0]
  diff_in_score = sum(unmatched_elements)
  return diff_in_score